In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib
import numba
from tqdm.notebook import tqdm
import os
import MetaTrader5 as mt5

cols = [
       'time',
       'bid',
       'ask',
       'type', # 注文タイプ
       'profit', # 実現損益
       'buy_position', # buyポジション数
       'buy_lot', # buyポジションのlot数合計
       'buy_price', # buyポジションの平均価格
       'buy_profit', # buyポジションの含み損益
       'sell_position', # sellポジション数
       'sell_lot', # sellポジションのlot数合計
       'sell_price', # sellポジションの平均価格
       'sell_profit' # sellポジションの含み損益
       ]

df_orders = pd.DataFrame(index=[], columns=cols)
first_lot = 0.01 # 初期ロット
pip_value = 135 # ピップ値
point = 0.01 # 価格の最小単位
spread_limit = 15 # 許容スプレッド
position_limit = 18 # 最大ポジション数
martin_factor =1.5 # マーチン倍率
nanpin_range = 200 # ナンピン幅
tp_range = 100 # 利確幅
lc_range = 1000 # ロスカット幅

@numba.njit
def calc_orders(
        first_lot,
        current_buy_lot,
        current_buy_price,
        current_sell_lot,
        current_sell_price,
        buy_position,
        buy_lot,
        buy_price,
        buy_profit,
        sell_position,
        sell_lot,
        sell_price,
        sell_profit,
        point,
        spread_limit,
        tick_len,
        tick_bid,
        tick_ask,
        ):

    orders_time = tick_bid.copy()
    orders_time[:] = np.nan

    orders_bid = tick_bid.copy()
    orders_bid[:] = np.nan

    orders_ask = tick_bid.copy()
    orders_ask[:] = np.nan

    orders_type = tick_bid.copy()
    orders_type[:] = np.nan

    orders_buy_position = tick_bid.copy()
    orders_buy_position[:] = np.nan

    orders_sell_position = tick_bid.copy()
    orders_sell_position[:] = np.nan

    orders_profit = tick_bid.copy()
    orders_profit[:] = np.nan

    orders_buy_profit = tick_bid.copy()
    orders_buy_profit[:] = np.nan

    orders_sell_profit = tick_bid.copy()
    orders_sell_profit[:] = np.nan

    orders_buy_lot = tick_bid.copy()
    orders_buy_lot[:] = np.nan

    orders_sell_lot = tick_bid.copy()
    orders_sell_lot[:] = np.nan

    orders_buy_price = tick_bid.copy()
    orders_buy_price[:] = np.nan

    orders_sell_price = tick_bid.copy()
    orders_sell_price[:] = np.nan

    if tick_len > 0:
        for i in range(tick_len):

            ##現在価格の取得
            bid = tick_bid[i]
            ask = tick_ask[i]
            spread = ask - bid

            ###entry_flgロジック
            if spread < spread_limit * point:
                entry_flg = 1
            else:
                entry_flg = 0

            ##ポジションの含み損益の確認
            if buy_lot == 0:
                buy_profit = 0
            else:
                buy_profit = round((bid - buy_price) * buy_lot * pip_value / 0.01, 0)
            if sell_lot == 0:
                sell_profit = 0
            else:
                sell_profit = round(-(ask - sell_price) * sell_lot * pip_value / 0.01, 0)

            ##ordersに出力
            orders_bid[i] = bid
            orders_ask[i] = ask
            orders_buy_position[i] = buy_position
            orders_buy_lot[i] = buy_lot
            orders_buy_price[i] = buy_price
            orders_buy_profit[i] = buy_profit
            orders_sell_position[i] = sell_position
            orders_sell_lot[i] = sell_lot
            orders_sell_price[i] = sell_price
            orders_sell_profit[i] = sell_profit
            #####

            ##新規buyエントリー
            if buy_position == 0 and entry_flg == 1:
                buy_position = 1 # buyポジション数
                current_buy_lot = first_lot # 最新のbuyポジションのlot数
                buy_lot = first_lot # buyポジションのlot数合計
                current_buy_price = ask # 最新のbuyポジション価格
                buy_price = ask # buyポジションの平均価格

                ##ordersに出力
                orders_type[i] = 1
                orders_buy_position[i] = buy_position
                orders_profit[i] = 0
                orders_buy_lot[i] = current_buy_lot
                orders_buy_price[i] = current_buy_price

            ##新規sellエントリー
            if sell_position == 0 and entry_flg == 1:
                sell_position = 1 # sellポジション数
                current_sell_lot = first_lot # 最新のsellポジションのlot数
                sell_lot = first_lot # sellポジションのlot数合計
                current_sell_price = bid # 最新のsellポジション価格
                sell_price = bid # sellポジションの平均価格

                ##ordersに出力
                orders_type[i] = 2
                orders_sell_position[i] = sell_position
                orders_profit[i] = 0
                orders_sell_lot[i] = current_sell_lot
                orders_sell_price[i] = current_sell_price

            ##追加buyエントリー
            if (
                buy_position > 0 and
                buy_position < position_limit and
                entry_flg == 1 and
                ask < current_buy_price - nanpin_range * point
                ):

                buy_position += 1 # buyポジション数
                x = buy_lot * buy_price # 平均価格算出用
                current_buy_lot = round(current_buy_lot * martin_factor + 0.001, 2) # 最新のbuyポジションのlot数
                buy_lot += current_buy_lot # buyポジションのlot数合計
                current_buy_price = ask # 最新のbuyポジション価格
                y = current_buy_lot * current_buy_price # 平均価格算出用
                buy_price = round(( x + y ) / buy_lot, 2) # buyポジションの平均価格

                ##ordersに出力
                orders_type[i] = 3
                orders_buy_position[i] = buy_position
                orders_profit[i] = 0
                orders_buy_lot[i] = current_buy_lot
                orders_buy_price[i] = current_buy_price

            ##追加sellエントリー
            if (
                sell_position > 0 and
                sell_position < position_limit and
                entry_flg == 1 and
                bid > current_sell_price + nanpin_range * point
                ):

                sell_position += 1 # sellポジション数
                x = sell_lot * sell_price # 平均価格算出用
                current_sell_lot = round(current_sell_lot * martin_factor + 0.001, 2) # 最新のsellポジションのlot数
                sell_lot += current_sell_lot # sellポジションのlot数合計
                current_sell_price = bid # 最新のsellポジション価格
                y = current_sell_lot * current_sell_price # 平均価格算出用
                sell_price = round(( x + y ) / sell_lot, 2) # sellポジションの平均価格

                ##ordersに出力
                orders_type[i] = 4
                orders_sell_position[i] = sell_position
                orders_profit[i] = 0
                orders_sell_lot[i] = current_sell_lot
                orders_sell_price[i] = current_sell_price

            ##buyクローズtp
            if buy_position >= 1 and bid > buy_price + tp_range * point / buy_position:

                ##ordersに出力
                orders_type[i] = 5
                orders_profit[i] = buy_profit
                #####

                buy_position = 0 # buyポジション数の初期化
                buy_profit = 0 # buy_profitの初期化
                current_buy_lot = 0 # 最新のbuyポジションのlot数の初期化
                buy_lot = 0 # buyポジションのlot数合計の初期化
                current_buy_price = 0 # 最新のbuyポジション価格の初期化
                buy_price = 0 # buyポジションの平均価格の初期化

            ##sellクローズtp
            if sell_position >= 1 and ask < sell_price - tp_range * point / sell_position:

                ##ordersに出力
                orders_type[i] = 6
                orders_profit[i] = sell_profit
                #####

                sell_position = 0 # sellポジション数の初期化
                sell_profit = 0 # sell_profitの初期化
                current_sell_lot = 0 # 最新のsellポジションのlot数の初期化
                sell_lot = 0 # sellポジションのlot数合計の初期化
                current_sell_price = 0 # 最新のsellポジション価格の初期化
                sell_price = 0 # sellポジションの平均価格の初期化
            ##buyクローズlc
            if buy_position >= position_limit and bid < current_buy_price - lc_range * point:

                ##ordersに出力
                orders_type[i] = 7
                orders_profit[i] = buy_profit
                #####

                buy_position = 0 # buyポジション数の初期化
                buy_profit = 0 # buy_profitの初期化
                current_buy_lot = 0 # 最新のbuyポジションのlot数の初期化
                buy_lot = 0 # buyポジションのlot数合計の初期化
                current_buy_price = 0 # 最新のbuyポジション価格の初期化
                buy_price = 0 # buyポジションの平均価格の初期化

            ##sellクローズlc
            if sell_position >= position_limit and ask > current_sell_price + lc_range*point:

                ##ordersに出力
                orders_type[i] = 8
                orders_profit[i] = sell_profit
                #####

                sell_position = 0 # sellポジション数の初期化
                sell_profit = 0 # sell_profitの初期化
                current_sell_lot = 0 # 最新のsellポジションのlot数の初期化
                sell_lot = 0 # sellポジションのlot数合計の初期化
                current_sell_price = 0 # 最新のsellポジション価格の初期化
                sell_price = 0 # sellポジションの平均価格の初期化

    return (orders_bid,
            orders_ask,
            orders_type,
            orders_profit,
            orders_buy_position,
            orders_buy_lot,
            orders_buy_price,
            orders_buy_profit,
            orders_sell_position,
            orders_sell_lot,
            orders_sell_price,
            orders_sell_profit,
            current_buy_lot,
            current_buy_price,
            current_sell_lot,
            current_sell_price,
            buy_position,
            buy_lot,
            buy_price,
            buy_profit,
            sell_position,
            sell_lot,
            sell_price,
            sell_profit)

def get_rates(symbol, frame, count):
    """ 一定の期間の時系列データを取得
       """
    rates = mt5.copy_rates_from_pos(symbol, frame, 0, count)
    df_rates = pd.DataFrame(rates)
    df_rates['time'] = pd.to_datetime(df_rates['time'], unit='s')
    df_rates = df_rates.set_index('time')
    df_rates.columns = [
        'Open', 'High', 'Low', 'Close', 'tick_volume', 'spread', 'real_volume',
    ]
    return df_rates

# MetaTrader 5パッケージについてのデータを表示する
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

# 指定された取引口座へのMetaTrader 5接続を確立
if not mt5.initialize(login=900013481, server="OANDA-Japan MT5 Live",password="Camar0924600"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

SYMBOL = "USDJPY"

# 時系列データを取得する
df_rates = get_rates(SYMBOL, frame=mt5.TIMEFRAME_H1, count=100)
last_close_price = df_rates['Close'][-1]
print(f"Close: {last_close_price}")


instrument = 'AUDJPY'
date_index = pd.date_range("2023-01-01", periods=31, freq="D")
AC_date_list = date_index.to_series().dt.strftime("%Y-%m-%d")
AC_date_list0 = AC_date_list[:5]
AC_date_list[-1]

YYYYMM = '202301'
filename = '/content/drive/My Drive/backup/'+instrument+'_M1_'+YYYYMM

df = pd.read_table(filename+'.csv')
df['time'] = pd.to_datetime(df['<DATE>']+ ' '+ df['<TIME>'])
df = df.set_index('time')
df = df.rename(columns={'<OPEN>':'open','<CLOSE>':'close','<HIGH>':'high','<LOW>':'low','<SPREAD>':'spread'})
df = df.fillna(method='ffill')
df.to_pickle(filename+'.pkl')

for AC_date in tqdm(AC_date_list):

    current_buy_lot=0.00
    current_buy_price=0.00
    current_sell_lot=0.00
    current_sell_price=0.00
    buy_position=0
    buy_lot=0.00
    buy_price=0.00
    buy_profit=0
    sell_position=0
    sell_lot=0.00
    sell_price=0.00
    sell_profit=0

    df = pd.read_pickle(filename+'.pkl')
    df = df[AC_date + ' 00:00:00' : AC_date + ' 23:59:59' ]
    #print(df)
    df_tick = df.reset_index()

    if(len(df_tick)>0):

        df = pd.DataFrame(index=[], columns=cols)

        (
        df['bid'], df['ask'], df['type'], df['profit'],
        df['buy_position'], df['buy_lot'], df['buy_price'], df['buy_profit'],
        df['sell_position'], df['sell_lot'], df['sell_price'], df['sell_profit'],
        current_buy_lot, current_buy_price,
        current_sell_lot, current_sell_price,
        buy_position, buy_lot, buy_price, buy_profit,
        sell_position, sell_lot, sell_price, sell_profit
        )= calc_orders(
            first_lot = first_lot,
            current_buy_lot = current_buy_lot,
            current_buy_price = current_buy_price,
            current_sell_lot = current_sell_lot,
            current_sell_price = current_sell_price,
            buy_position = buy_position,
            buy_lot = buy_lot,
            buy_price = buy_price,
            buy_profit = buy_profit,
            sell_position = sell_position,
            sell_lot = sell_lot,
            sell_price = sell_price,
            sell_profit = sell_profit,
            point = point,
            spread_limit = spread_limit,
            tick_len = len(df_tick),
            tick_bid = df_tick['open'].values,
            #tick_ask = df_tick['ask'].values,
            tick_ask = df_tick['open'].values
            )

        df['time'] = df_tick['time']
        df['spread'] = df_tick['spread']
        df['pip_value'] = pip_value

        #df = df.dropna( )
        #df.to_pickle('/content/drive/My Drive/backup/outputs/df_orders_'+AC_date+'.pkl')
        df.to_csv('/content/drive/My Drive/backup/outputs/df_orders_'+AC_date+'.csv')

In [8]:
import pandas as pd

pd.DataFrame(index=[],columns=['a','b','c'])

,a,b,c
